In [39]:
import ee
import geemap
import numpy as np
import sys
sys.path.insert(1, '/Users/gopal/Projects/ml/downloadGEErasters')
import rs
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [3]:
ee.Initialize()

In [281]:
ee_geometry_box = ee.Geometry.Polygon(
        [[[77.42201334186198, 11.398136848502652],
          [77.42201334186198, 10.873987098476817],
          [78.33387857623698, 10.873987098476817],
          [78.33387857623698, 11.398136848502652]]], None, False)

In [257]:
ee_geometry = ee.FeatureCollection("users/gopalpenny/cauvery/Cauvery_boundary5")
def prep_oli8_ic(ee_geometry):
    """ Prepare Landsat 8 image collection
    This in
    """
    
    oli8_output_bands = ['SR_B7','SR_B6','SR_B5','SR_B4','SR_B3','SR_B2','clouds','shadows','clouds_shadows']
    oli8_ic = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
      .filterBounds(ee_geometry)
      
    get_qaband_clouds_shadows = rs.get_qaband_clouds_shadows_func(
          qa_bandname = 'QA_PIXEL', 
          cloud_bit = 3, 
          shadow_bit = 4,
          keep_orig_bands = True) 
    oli8_clouds_ic = (oli8_ic
      .map(get_qaband_clouds_shadows))
      # .map(lambda im: im.addBands(im.expression('im.clouds | im.clouds_shadows', {'im' : im}).rename('cloudmask'))))
      
    return oli8_clouds_ic.select(oli8_output_bands)
    # Get oli8 pixel timeseries

In [95]:

# oli8_season = oli8.filterDate("2014-06-01","2015-05-31")

'2016-06-01'

In [258]:
oli8 = prep_oli8_ic(ee_geometry) \
  .filterMetadata('CLOUD_COVER', 'less_than', 50)



In [259]:
def add_ndvi_band(im):
    im = im.addBands(im.normalizedDifference(['SR_B5','SR_B4']).rename('NDVI'))
    im = im.addBands(im.normalizedDifference(['SR_B3','SR_B6']).rename('MNDWI'))
    im = im.updateMask(im.select(['clouds_shadows']).eq(0))
    im = im.float()
    return im

oli8_bands = oli8.map(add_ndvi_band)
oli8_bands.size().getInfo()

oli8_bands.filterDate(rabi_start_date, summer_start_date).size().getInfo()

# oli8_ndvi.first().getInfo()

43

In [260]:
def season_median(ic, start_date, end_date, season_name):
    
    # print('start_date',start_date)
    # print('end_date',end_date)
    band_names_orig = ['SR_B5','SR_B4','SR_B3','SR_B2','NDVI','MNDWI']
    band_names_median = [name + '_median' for name in band_names_orig]
    band_names_season = [name + '_' + season_name for name in band_names_orig]
    ic = ic \
      .filterDate(start_date, end_date)
    num_images = ic.size().getInfo()
    print(season_name + ' num images:',num_images)
    
    oli8_median = ic \
      .reduce(ee.Reducer.median()) \
      .select(band_names_median, band_names_season)
    
    return oli8_median

In [261]:


def get_im_seasons(ic, monsoon_year):
    
    # specify the dates for each of the seasons
    kharif_start_dt = datetime.strptime(str(monsoon_year) + "-06-01", "%Y-%m-%d")
    kharif_start_date = datetime.strftime(kharif_start_dt, "%Y-%m-%d")
    rabi_start_date = datetime.strftime(kharif_start_dt + relativedelta(months = 4), "%Y-%m-%d")
    summer_start_date = datetime.strftime(kharif_start_dt + relativedelta(months = 8), "%Y-%m-%d")
    summer_end_date = datetime.strftime(kharif_start_dt + relativedelta(months = 12), "%Y-%m-%d")

    # calculate the median bands for each of the seasons
    im_kharif = season_median(ic, kharif_start_date, rabi_start_date, 'kharif')
    im_rabi = season_median(ic, rabi_start_date, summer_start_date, 'rabi')
    im_summer = season_median(ic, summer_start_date, summer_end_date, 'summer')
    
    # combine the bands from each season to a single image
    im_seasons = im_kharif \
      .addBands(im_rabi) \
      .addBands(im_summer) \
      .set('monsoon_year', monsoon_year)
    
    im_seasons = im_seasons.addBands(
        im_seasons.expression('(im.NDVI_kharif > 0.25) + (im.NDVI_rabi > 0.25) + (im.NDVI_summer > 0.25)', {'im':im_seasons}).rename('crops')
    )

    return im_seasons

In [262]:
# monsoon_year = 2015
im_seasons_15 = get_im_seasons(oli8_bands, 2015)
im_seasons_16 = get_im_seasons(oli8_bands, 2016)
im_seasons_17 = get_im_seasons(oli8_bands, 2017)
im_seasons_18 = get_im_seasons(oli8_bands, 2018)
im_seasons_19 = get_im_seasons(oli8_bands, 2019)
im_seasons_20 = get_im_seasons(oli8_bands, 2020)
im_seasons_21 = get_im_seasons(oli8_bands, 2021)

kharif num images: 21
rabi num images: 43
summer num images: 68
kharif num images: 30
rabi num images: 50
summer num images: 67
kharif num images: 24
rabi num images: 54
summer num images: 57
kharif num images: 23
rabi num images: 55
summer num images: 68
kharif num images: 29
rabi num images: 48
summer num images: 62
kharif num images: 24
rabi num images: 40
summer num images: 62
kharif num images: 30
rabi num images: 40
summer num images: 60


In [ ]:
im_seasons_all = ee.ImageCollection([im_seasons_15, 
                                     im_seasons_16,
                                     im_seasons_17,
                                     im_seasons_18,
                                     im_seasons_19,
                                     im_seasons_20,
                                     im_seasons_21])
im_seasons_all.size().getInfo()

In [279]:
def get_crops_bit_adj(im):
    monsoon_year = im.get('monsoon_year')
    crops_year_diff = ee.Number(monsoon_year).subtract(2015)
    crops_mult = ee.Number(4).pow(crops_year_diff)
    
    return im.select(['crops']).multiply(crops_mult)
    # crops_bit = 
    
    
crops_2bit_base2015 = im_seasons_all \
  .filterMetadata('monsoon_year','lte',2018)
  .map(get_crops_bit_adj) \
  .reduce(ee.Reducer.sum())
    

In [284]:
def select_crops_year(im):
    monsoon_year = ee.String(im.get('monsoon_year'))
    band_name = ee.String('y').cat(monsoon_year)
    
    return im.select(['crops'],[band_name])
    

crops_bands = im_seasons_all.select(['crops']).toBands()

crops_bands.getInfo()

{'type': 'Image',
 'bands': [{'id': '0_crops',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 3},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': '1_crops',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 3},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': '2_crops',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 3},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': '3_crops',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 3},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': '4_crops',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 3},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': '5_crops',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 3},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 

In [ ]:
mall = geemap.Map()

mall.addLayer(crops_2bit_base2015,{'min':0,'max':16000},'crops_2bit_base2015')
mall.centerObject(ee_geometry_box, 10)
mall

In [288]:
crops_task = ee.batch.Export.image.toDrive(
    image = crops_bands,
    description = 'crops_bands',
    folder = 'classy_downloads',
    fileNamePrefix = 'karur_crops_2015_2021',
    region = ee_geometry_box,
    scale = 30)

crops_task.start()

In [ ]:
ee.batch.Export.image

In [250]:
im_seasons_16 = im_seasons_16.addBands(
    im_seasons_16.expression('(im.NDVI_kharif > 0.25) + (im.NDVI_rabi > 0.25) + (im.NDVI_summer > 0.25)', {'im':im_seasons_16}).rename('crops')
)

im_seasons_16

In [234]:


im_seasons_16

In [ ]:
monsoon_years = [2015, 2016, 2017, 2018, 2019, 2020]

monsoon_years = ee.List.sequence(2015, 2020)

def get_oli8_seasons(monsoon_year):
    return get_im_seasons(oli8_bands, monsoon_year)

oli8_seasons = ee.ImageCollection(monsoon_years.map(get_oli8_seasons))

# print(oli8_seasons.size().getInfo())
# if not 'im_years' in locals():
# #     im_years = {}
# for monsoon_year in monsoon_years:
#     yname = 'y' + str(monsoon_year)
#     # print(yname)
#     if not yname in im_years:
#         print(yname)
#         im_seasons[yname] = 


In [ ]:
oli8.filter(ee.Filter.dateRange().getInfo()

In [17]:
oli8_geom.first().getInfo()

1506850

In [53]:
m.addLayer(oli8.first(), {'bands':['SR_B5','SR_B4','SR_B3'], 'min':5000, 'max':30000},'oli8')


In [264]:
# gfsad30 = ee.Image("users/gopalpenny/GFSAD30")
m = geemap.Map()

m.addLayer(im_seasons_15.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3},'crops 2015')
m.addLayer(im_seasons_16.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3},'crops 2016')
m.addLayer(im_seasons_17.clip(ee_geometry), {'bands':['crops'], 'min':0, 'max':3},'crops 2017')
# m.centerObject(ee_geometry, 13)
m.addLayerControl()
m

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [290]:
m16 = geemap.Map()

# m.addLayer(gfsad30, {'min':0, 'max':2},'gfsad30')
# m16.addLayer(im_seasons_16, {'bands':['SR_B5_kharif','SR_B4_kharif','SR_B3_kharif'], 'min':5000, 'max':30000},'oli8 kharif')
# m16.addLayer(im_seasons_16, {'bands':['SR_B5_rabi','SR_B4_rabi','SR_B3_rabi'], 'min':5000, 'max':30000},'oli8 rabi')
m16.addLayer(im_seasons_16, {'bands':['NDVI_kharif'], 'min':0, 'max':0.5},'ndvi kharif')
m16.addLayer(im_seasons_16, {'bands':['NDVI_rabi'], 'min':0, 'max':0.5},'ndvi rabi')
m16.addLayer(im_seasons_16, {'bands':['NDVI_summer'], 'min':0, 'max':0.5},'ndvi summer')
# m16.addLayer(im_seasons_16.gt(0.25), {'bands':['NDVI_rabi'], 'min':0, 'max':0.5},'crop rabi')
# m16.addLayer(im_seasons_15, {'bands':['crops'], 'min':0, 'max':3},'crops 2015')
# m16.addLayer(im_seasons_16, {'bands':['crops'], 'min':0, 'max':3},'crops 2016')
# m16.addLayer(im_seasons_16, {'bands':['crops'], 'min':0, 'max':3},'crops 2016')
# m16.addLayer(im_seasons_16, {'bands':['MNDWI_kharif','NDVI_rabi','NDVI_summer'], 'min':0.1, 'max':0.3},'mndwi kharif')
# m16.addLayer(im_seasons_16, {'bands':['MNDWI_kharif'], 'min':-0.2, 'max':0.1},'mndwi kharif')
# m16.addLayer(im_seasons_16, {'bands':['MNDWI_rabi'], 'min':-0.2, 'max':0.1},'mndwi rabi')
# m.addLayer(oli8_summer, {'bands':['NDVI'], 'min':0, 'max':0.5},'ndvi summer')
# m.addLayer(oli8_kharif, {'bands':['MNDWI'], 'min':-0.3, 'max':0},'ndwi_median')
# m.addLayer(ee_geometry)
m16.centerObject(ee_geometry, 13)
m16.addLayerControl()
m16

Map(center=[11.748174793282523, 77.4497503999458], controls=(WidgetControl(options=['position', 'transparent_b…

In [115]:
summer_start_date

'2016-02-01'

In [191]:
testdict = {}
testdict['1'] = 3
# testdict['0'] == None
im_years

{}